In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'    
wikitables =pd.read_html(page, index_col=0, attrs={"class":"wikitable"})     
wikitables[0].to_csv('canada.csv') #Just the first table is important

In [3]:
df_can=pd.read_csv('canada.csv', header=1) 

In [4]:

df_can.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df=df_can[df_can.Borough!="Not assigned"].reset_index()
del df['index']

In [6]:
df.sort_values('Postcode')# We sort the values to get all the same Postalcodes together
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
#auxiliary variables
pc_aux=''
nb_aux=''
#We use a for to check the df and get the index
for i in df.index:
    #We store de Postcode and neighbourhood of the row i
    pc=df['Postcode'][i]
    nb=df['Neighbourhood'][i]
    #We see if the value is a not assigned value, if it is we modify the value
    if nb=='Not assigned':
        df['Neighbourhood'][i]=df['Borough'][i]
        nb=df['Neighbourhood'][i]
    #We see if the postal code is the same as the previous one, if it is we modify the value and drop the duplicates 
    if pc==pc_aux:
        df['Neighbourhood'][i-1]=nb_aux+','+' '+nb
        df.drop(i, axis=0, inplace=True) 
        nb=nb_aux+','+' '+nb
    #update auxiliari variables
    pc_aux=pc
    nb_aux=nb
    
df=df.reset_index()
del df['index']
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
df.shape

(103, 3)

In [9]:
df_l=pd.read_csv("http://cocl.us/Geospatial_data")

In [10]:
df_l.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df['Latitude']=''
df['Longitude']=''
df.add('Latitude')
for i in df.index:
    for j in df_l.index:
        if df['Postcode'][i]==df_l['Postal Code'][j]:
            df['Latitude'][i]=df_l['Latitude'][j]
            df['Longitude'][i]=df_l['Longitude'][j]

df.head(12)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
7,M3B,North York,Don Mills North,43.7459,-79.3522
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7064,-79.3099
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3789
